# Import dependencies

In [1]:
import os
import re
import pickle
import numpy as np
import pandas as pd
import importlib

import spacy

from utilities import _support
from utilities._support import ig_profiles

from nltk.tokenize import word_tokenize, sent_tokenize

importlib.reload(_support)

<module 'utilities._support' from 'c:\\Users\\tommaso\\Documents\\_Projects\\IGScraper\\utilities\\_support.py'>

# Concat CSVs from same profile

In [2]:
indagato = ig_profiles['salvini']

csv_path = f'csv/{indagato}/'
csv_files = os.listdir(csv_path)

In [3]:
postfiles = list(set([csv for csv in csv_files if csv.startswith(indagato) and csv.endswith('posts.csv')]))
# commentfiles = list(set([csv for csv in csv_files if csv.startswith(indagato) and csv.endswith('comments.csv')]))

# assert(len(postfiles) == len(commentfiles))

In [4]:
files = postfiles
column = 'description'

df = pd.concat([pd.read_csv(csv_path+fn, sep='\t') for fn in files]).reset_index(drop=True)

In [5]:
print(f'shape: {df.shape}')
df.head()

shape: (1095, 4)


,post_id,author,description,date
0,Ck9MRyyKIT5,matteosalviniofficial,"“Non reagiva, non rispondeva agli stimoli, il cuore era fermo”. Marika si laurea in fisioterapia, festeggia e mentre torna a casa, travestita da lottatrice di sumo, si accorge che in fondo al bus un uomo si accascia: lo soccorre e poi applica alla lettera le procedure salva-vita. Complimenti, Marika, per il coraggio e la prontezza, testimonianza di come sia fondamentale conoscere e saper applicare le manovre di primo soccorso. Sii orgogliosa di te stessa, grazie. ❤️",2022-11-14T21:16:37.000Z
1,Ck8z_X6q8zg,matteosalviniofficial,"GRAZIE, dialogare con voi e ascoltare le vostre opinioni non è per me un diritto, ma un dovere!",2022-11-14T17:44:23.000Z
2,Ck8UuspKY5c,matteosalviniofficial,"Bollette, lavoro, pensioni, tasse, sicurezza, immigrazione. Primi giorni al governo molto positivi, anche secondo molti di voi. Grazie per la fiducia, avanti cosi! 🇮🇹",2022-11-14T13:11:14.000Z
3,Ck8MdbxKjKk,matteosalviniofficial,"Paderno Dugnano, all'inaugurazione del prolungamento della Tangenziale Nord di Milano, un’opera fondamentale per la viabilità milanese e che durante la realizzazione ha dato lavoro a più di mille persone, avanti così! Viva l'Italia che corre, che costruisce e che investe nelle Infrastrutture 🇮🇹",2022-11-14T11:58:58.000Z
4,Ck6mCEsKaFN,matteosalviniofficial,Buona vita Nicoletta!❤️,2022-11-13T21:03:56.000Z


In [6]:
df.to_csv(f'csv/{indagato}/{indagato}_posts_20230316_0-1100.csv', sep='\t', index=False)

# Load data

In [17]:
df = pd.read_csv(f'csv//{indagato}/{indagato}_posts_20230316_0-1100.csv', sep='\t')

In [18]:
df

,post_id,author,description,date
0,Ck9MRyyKIT5,matteosalviniofficial,"“Non reagiva, non rispondeva agli stimoli, il cuore era fermo”. Marika si laurea in fisioterapia, festeggia e mentre torna a casa, travestita da lottatrice di sumo, si accorge che in fondo al bus un uomo si accascia: lo soccorre e poi applica alla lettera le procedure salva-vita. Complimenti, Marika, per il coraggio e la prontezza, testimonianza di come sia fondamentale conoscere e saper applicare le manovre di primo soccorso. Sii orgogliosa di te stessa, grazie. ❤️",2022-11-14T21:16:37.000Z
1,Ck8z_X6q8zg,matteosalviniofficial,"GRAZIE, dialogare con voi e ascoltare le vostre opinioni non è per me un diritto, ma un dovere!",2022-11-14T17:44:23.000Z
2,Ck8UuspKY5c,matteosalviniofficial,"Bollette, lavoro, pensioni, tasse, sicurezza, immigrazione. Primi giorni al governo molto positivi, anche secondo molti di voi. Grazie per la fiducia, avanti cosi! 🇮🇹",2022-11-14T13:11:14.000Z
3,Ck8MdbxKjKk,matteosalviniofficial,"Paderno Dugnano, all'inaugurazione del prolungamento della Tangenziale Nord di Milano, un’opera fondamentale per la viabilità milanese e che durante la realizzazione ha dato lavoro a più di mille persone, avanti così! Viva l'Italia che corre, che costruisce e che investe nelle Infrastrutture 🇮🇹",2022-11-14T11:58:58.000Z
4,Ck6mCEsKaFN,matteosalviniofficial,Buona vita Nicoletta!❤️,2022-11-13T21:03:56.000Z
...,...,...,...,...
1090,ClQWEQzqj7V,matteosalviniofficial,"Grande segretario, super ministro, ottimo governatore, leghista sempre e per sempre. Buon vento Roberto.",2022-11-22T07:47:43.000Z
1091,ClPUZ2BqWb9,matteosalviniofficial,"Più di 12.000 voti in poche ore al sondaggio che ho pubblicato nelle storie Instagram, col 93% di voi che si dice d’accordo. Bene così.",2022-11-21T22:13:57.000Z
1092,ClOsD9DsuEu,matteosalviniofficial,#colpadiSalvini? 🤣,2022-11-21T16:21:27.000Z
1093,ClOdlncqeZ-,matteosalviniofficial,"Genga (Ancona), bullo senza biglietto si scaglia contro il controllore e gli rompe il naso. Questo delinquente non può passarla liscia!",2022-11-21T14:15:50.000Z


# Utils

In [16]:
def lambda_pipe(element, lpipe):
    return lpipe[0](element) if len(lpipe)==1 else lambda_pipe(lpipe[0](element), lpipe[1:])

# Data preparation

## Text preprocess

- lower
- no urls
- no non-alpha chars:
    - no @ tag
    - no punctuation (except '.')
    - no emoji
    - no numbers
    - no escape chars
- no extra spaces
- no extra '.'

In [21]:
lower = lambda s: s.lower()
no_urls = lambda s: re.sub(r'https?://\S+|www.\.\S+', '', s)
no_nonalpha = lambda s: ''.join([c for c in s if c.isalpha() or c==' ' or c=='.'])
no_ellipses = lambda s: re.sub(r'\.{2,}', '. ', s)
no_extra_spaces = lambda s: ' '.join(s.split())

def prep_df(df, column_name=f'{column}'):
    df[f'{column}_prep'] = df[column_name].apply(lambda c: lambda_pipe(c, [lower, no_urls, no_nonalpha, no_ellipses, no_extra_spaces]))
    return df

In [22]:
df = prep_df(df)

df.head(2)

,post_id,author,description,date,description_prep
0,Ck9MRyyKIT5,matteosalviniofficial,"“Non reagiva, non rispondeva agli stimoli, il cuore era fermo”. Marika si laurea in fisioterapia, festeggia e mentre torna a casa, travestita da lottatrice di sumo, si accorge che in fondo al bus un uomo si accascia: lo soccorre e poi applica alla lettera le procedure salva-vita. Complimenti, Marika, per il coraggio e la prontezza, testimonianza di come sia fondamentale conoscere e saper applicare le manovre di primo soccorso. Sii orgogliosa di te stessa, grazie. ❤️",2022-11-14T21:16:37.000Z,non reagiva non rispondeva agli stimoli il cuore era fermo. marika si laurea in fisioterapia festeggia e mentre torna a casa travestita da lottatrice di sumo si accorge che in fondo al bus un uomo si accascia lo soccorre e poi applica alla lettera le procedure salvavita. complimenti marika per il coraggio e la prontezza testimonianza di come sia fondamentale conoscere e saper applicare le manovre di primo soccorso. sii orgogliosa di te stessa grazie.
1,Ck8z_X6q8zg,matteosalviniofficial,"GRAZIE, dialogare con voi e ascoltare le vostre opinioni non è per me un diritto, ma un dovere!",2022-11-14T17:44:23.000Z,grazie dialogare con voi e ascoltare le vostre opinioni non è per me un diritto ma un dovere


## Tokenization

1. Text → Sentences
2. Senteces → Words

In [23]:
tokenize_text2senteces = lambda text: [sentence[:-1] if sentence.endswith('.') else sentence for sentence in sent_tokenize(text)]
tokenize_senteces2words = lambda sentences: [word_tokenize(sentence) if type(sentence) is str and len(sentence)>0 else None for sentence in sentences]

def tokenize_df(df, column_name=f'{column}_prep'):
    df[f'{column}_token'] = df[column_name].apply(lambda c: lambda_pipe(c, [tokenize_text2senteces, lambda x: list(x), tokenize_senteces2words]))
    return df

In [26]:
df = tokenize_df(df)

In [27]:
df

,post_id,author,description,date,description_prep,description_token
0,Ck9MRyyKIT5,matteosalviniofficial,"“Non reagiva, non rispondeva agli stimoli, il cuore era fermo”. Marika si laurea in fisioterapia, festeggia e mentre torna a casa, travestita da lottatrice di sumo, si accorge che in fondo al bus un uomo si accascia: lo soccorre e poi applica alla lettera le procedure salva-vita. Complimenti, Marika, per il coraggio e la prontezza, testimonianza di come sia fondamentale conoscere e saper applicare le manovre di primo soccorso. Sii orgogliosa di te stessa, grazie. ❤️",2022-11-14T21:16:37.000Z,non reagiva non rispondeva agli stimoli il cuore era fermo. marika si laurea in fisioterapia festeggia e mentre torna a casa travestita da lottatrice di sumo si accorge che in fondo al bus un uomo si accascia lo soccorre e poi applica alla lettera le procedure salvavita. complimenti marika per il coraggio e la prontezza testimonianza di come sia fondamentale conoscere e saper applicare le manovre di primo soccorso. sii orgogliosa di te stessa grazie.,"[[“, Non, reagiva, ,, non, rispondeva, agli, stimoli, ,, il, cuore, era, fermo, ”], [Marika, si, laurea, in, fisioterapia, ,, festeggia, e, mentre, torna, a, casa, ,, travestita, da, lottatrice, di, sumo, ,, si, accorge, che, in, fondo, al, bus, un, uomo, si, accascia, :, lo, soccorre, e, poi, applica, alla, lettera, le, procedure, salva-vita], [Complimenti, ,, Marika, ,, per, il, coraggio, e, la, prontezza, ,, testimonianza, di, come, sia, fondamentale, conoscere, e, saper, applicare, le, manovre, di, primo, soccorso], [Sii, orgogliosa, di, te, stessa, ,, grazie], [❤️]]"
1,Ck8z_X6q8zg,matteosalviniofficial,"GRAZIE, dialogare con voi e ascoltare le vostre opinioni non è per me un diritto, ma un dovere!",2022-11-14T17:44:23.000Z,grazie dialogare con voi e ascoltare le vostre opinioni non è per me un diritto ma un dovere,"[[GRAZIE, ,, dialogare, con, voi, e, ascoltare, le, vostre, opinioni, non, è, per, me, un, diritto, ,, ma, un, dovere, !]]"
2,Ck8UuspKY5c,matteosalviniofficial,"Bollette, lavoro, pensioni, tasse, sicurezza, immigrazione. Primi giorni al governo molto positivi, anche secondo molti di voi. Grazie per la fiducia, avanti cosi! 🇮🇹",2022-11-14T13:11:14.000Z,bollette lavoro pensioni tasse sicurezza immigrazione. primi giorni al governo molto positivi anche secondo molti di voi. grazie per la fiducia avanti cosi,"[[Bollette, ,, lavoro, ,, pensioni, ,, tasse, ,, sicurezza, ,, immigrazione], [Primi, giorni, al, governo, molto, positivi, ,, anche, secondo, molti, di, voi], [Grazie, per, la, fiducia, ,, avanti, cosi, !], [🇮🇹]]"
3,Ck8MdbxKjKk,matteosalviniofficial,"Paderno Dugnano, all'inaugurazione del prolungamento della Tangenziale Nord di Milano, un’opera fondamentale per la viabilità milanese e che durante la realizzazione ha dato lavoro a più di mille persone, avanti così! Viva l'Italia che corre, che costruisce e che investe nelle Infrastrutture 🇮🇹",2022-11-14T11:58:58.000Z,paderno dugnano allinaugurazione del prolungamento della tangenziale nord di milano unopera fondamentale per la viabilità milanese e che durante la realizzazione ha dato lavoro a più di mille persone avanti così viva litalia che corre che costruisce e che investe nelle infrastrutture,"[[Paderno, Dugnano, ,, all'inaugurazione, del, prolungamento, della, Tangenziale, Nord, di, Milano, ,, un, ’, opera, fondamentale, per, la, viabilità, milanese, e, che, durante, la, realizzazione, ha, dato, lavoro, a, più, di, mille, persone, ,, avanti, così, !], [Viva, l'Italia, che, corre, ,, che, costruisce, e, che, investe, nelle, Infrastrutture, 🇮🇹]]"
4,Ck6mCEsKaFN,matteosalviniofficial,Buona vita Nicoletta!❤️,2022-11-13T21:03:56.000Z,buona vita nicoletta,"[[Buona, vita, Nicoletta, !, ❤️]]"
...,...,...,...,...,...,...
1090,ClQWEQzqj7V,matteosalviniofficial,"Grande segretario, super ministro, ottimo governatore, leghista sempre e per sempre. Buon vento Roberto.",2022-11-22T07:47:43.000Z,grande segretario super ministro ottimo

## Lemmatization + POS tagging

Using spacy nlp model <br>
[ https://spacy.io/models/it#it_core_news_md ]

- Lemmatization
    - reduction to base form
- POS tag filtering:
    - nouns
    - verbs
    - adjectives
    - adverbs

In [28]:
spacy_nlp = spacy.load("it_core_news_md", disable=['ner', 'entity_linker', 'entity_ruler',
                                                    'textcat', 'textcat_multilabel', 'morphologizer', 
                                                    'senter', 'sentencizer', 'transformer'])

sentence2doc = lambda sentence: spacy_nlp(' '.join(sentence) if type(sentence) is list else sentence) if sentence!=None else None
lemmatize_doc = lambda doc: [t.lemma_ if t != None else None for t in doc] if doc!=None else None
postag_doc = lambda doc: [t.tag_ if t != None else None for t in doc] if doc!=None else None

def lemma_tagger_df(df, column_name=f'{column}_token'):
    docs = df[column_name].apply(lambda c: [sentence2doc(sentence) for sentence in c])
    df[f'{column}_lemma'] = docs.apply(lambda docs: [lemmatize_doc(sentence) for sentence in docs])
    df[f'{column}_postag'] = docs.apply(lambda docs: [postag_doc(sentence) for sentence in docs])
    return df

c:\Users\tommaso\AppData\Local\Programs\Python\Python39\lib\site-packages\spacy\util.py:877: UserWarning: [W095] Model 'it_core_news_md' (3.2.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.5.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [29]:
df = lemma_tagger_df(df)

In [15]:
df

,post_id,author,description,date,description_prep,description_token,description_lemma,description_postag
0,Ck9MRyyKIT5,matteosalviniofficial,"“Non reagiva, non rispondeva agli stimoli, il cuore era fermo”. Marika si laurea in fisioterapia, festeggia e mentre torna a casa, travestita da lottatrice di sumo, si accorge che in fondo al bus un uomo si accascia: lo soccorre e poi applica alla lettera le procedure salva-vita. Complimenti, Marika, per il coraggio e la prontezza, testimonianza di come sia fondamentale conoscere e saper applicare le manovre di primo soccorso. Sii orgogliosa di te stessa, grazie. ❤️",2022-11-14T21:16:37.000Z,non reagiva non rispondeva agli stimoli il cuore era fermo. marika si laurea in fisioterapia festeggia e mentre torna a casa travestita da lottatrice di sumo si accorge che in fondo al bus un uomo si accascia lo soccorre e poi applica alla lettera le procedure salvavita. complimenti marika per il coraggio e la prontezza testimonianza di come sia fondamentale conoscere e saper applicare le manovre di primo soccorso. sii orgogliosa di te stessa grazie.,"[[“, Non, reagiva, ,, non, rispondeva, agli, stimoli, ,, il, cuore, era, fermo, ”], [Marika, si, laurea, in, fisioterapia, ,, festeggia, e, mentre, torna, a, casa, ,, travestita, da, lottatrice, di, sumo, ,, si, accorge, che, in, fondo, al, bus, un, uomo, si, accascia, :, lo, soccorre, e, poi, applica, alla, lettera, le, procedure, salva-vita], [Complimenti, ,, Marika, ,, per, il, coraggio, e, la, prontezza, ,, testimonianza, di, come, sia, fondamentale, conoscere, e, saper, applicare, le, manovre, di, primo, soccorso], [Sii, orgogliosa, di, te, stessa, ,, grazie], [❤️]]","[[“], [n], [o], [n], [ ], [r], [e], [a], [g], [il], [v], [a], [,], [ ], [n], [o], [n], [ ], [r], [il], [s], [p], [o], [n], [d], [e], [v], [a], [ ], [a], [g], [l], [il], [ ], [s], [t], [il], [m], [o], [l], [il], [,], [ ], [il], [l], [ ], [c], [u], [o], [r], [e], [ ], [e], [r], [a], [ ], [f], [e], [r], [m], [o], [”], [.], [ ], [m], [a], [r], [il], [k], [a], [ ], [s], [il], [ ], [l], [a], [u], [r], [e], [a], [ ], [il], [n], [ ], [f], [il], [s], [il], [o], [t], [e], [r], [a], [p], [il], [a], [,], [ ], [f], [e], ...]","[[FB], [X], [CC], [X], [_SP], [SP], [CC], [E], [X], [RD], [NO], [E], [FF], [_SP], [X], [CC], [X], [_SP], [SP], [RD], [X], [SP], [CC], [X], [S], [CC], [NO], [E], [_SP], [E], [X], [RD], [RD], [_SP], [X], [FB], [RD], [S], [CC], [RD], [RD], [FF], [_SP], [RD], [RD], [_SP], [S], [RD], [CC], [SP], [CC], [_SP], [CC], [SP], [E], [_SP], [X], [CC], [SP], [S], [CC], [FB], [FS], [_SP], [SP], [E], [SP], [RD], [X], [E], [_SP], [X], [RD], [_SP], [RD], [E], [RD], [SP], [CC], [E], [_SP], [RD], [X], [_SP], [X], [RD], [X], [RD], [CC], [FB], [CC], [SP], [E], [SP], [RD], [E], [FF], [_SP], [X], [CC], ...]"
1,Ck8z_X6q8zg,matteosalviniofficial,"GRAZIE, dialogare con voi e ascoltare le vostre opinioni non è per me un diritto, ma un dovere!",2022-11-14T17:44:23.000Z,grazie dialogare con voi e ascoltare le vostre opinioni non è per me un diritto ma un dovere,"[[GRAZIE, ,, dialogare, con, voi, e, ascoltare, le, vostre, opinioni, non, è, per, me, un, diritto, ,, ma, un, dovere, !]]","[[g], [r], [a], [z], [il], [e], [,], [ ], [d], [il], [a], [l], [o], [g], [a], [r], [e], [ ], [c], [o], [n], [ ], [v], [o], [il], [ ], [e], [ ], [a], [s], [c], [o], [l], [t], [a], [r], [e], [ ], [l], [e], [ ], [v], [o], [s], [t], [r], [e], [ ], [o], [p], [il], [n], [il], [o], [n], [il], [ ], [n], [o], [n], [ ], [essere], [ ], [p], [e], [r], [ ], [m], [e], [ ], [u], [n], [ ], [d], [il], [r], [il], [t], [t], [o], [,], [ ], [m], [a], [ ], [u], [n], [ ], [d], [o], [v], [e], [r], [e], [!]]","[[SP], [SP], [E], [SP], [RD], [CC], [FF], [_SP], [S], [RD], [E], [RD], [CC], [X], [E], [SP], [CC], [_SP], [S], [CC], [X], [_SP], [NO], [CC], [RD], [_SP], [CC], [_SP], [E], [X], [S], [CC], [RD], [FB], [E], [SP], [CC], [_SP], [RD], [CC], [_SP], [NO], [CC], [X], [FB], [SP], [CC], [_SP], [CC], [SP], [RD], [X], [RD], [CC], [X], [RD], [_SP], [X], [C

# Save CSV

## Lookup users

In [261]:
with open('utilities/lookup_users.pkl', 'rb') as f:
    full_lookup = pickle.load(f)

this_users = set(df.user)
new_users = set(full_lookup.keys()) - this_users

In [262]:
start_idx = len(full_lookup)
for i,new_user in enumerate(new_users):
    full_lookup[new_users] = f'user_{str(start_idx+i)}'

In [263]:
with open('utilities/lookup_users.pkl', 'wb') as f:
    pickle.dump(full_lookup, f)

In [265]:
df.user = df.user.apply(lambda u: full_lookup[u])

## Write CSV as json

In [271]:
fname = 'gitshared_csv/matteosalviniofficial_posts_20230316_0-1100_preprocessed.json'

In [272]:
df.to_json(fname)